## This notebook
- uses the `hwwprocessor` processor which doesn't apply any cuts but saves the cuts as bools for us to test and cross-check
- runs over EGamma sample... one root file...

In [1]:
import json
import uproot
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from coffea import nanoevents
from coffea import processor
import time

import argparse
import warnings
import pyarrow as pa
import pyarrow.parquet as pq
import pickle as pkl
import pandas as pd
import os
import sys
sys.path.append("../")

from collections import defaultdict
import pickle as pkl
import pyarrow as pa
import awkward as ak
import numpy as np
import pandas as pd
import json
import os
import shutil
import pathlib
from typing import List, Optional
import pyarrow.parquet as pq

from coffea import processor
from coffea.nanoevents.methods import candidate, vector
from coffea.analysis_tools import Weights, PackedSelection
from boostedhiggs.utils import match_HWW
from boostedhiggs.btag import btagWPs
from boostedhiggs.btag import BTagCorrector

import hist as hist2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import mplhep as hep
from hist.intervals import clopper_pearson_interval

import warnings
warnings.filterwarnings("ignore", message="Found duplicate branch ")
warnings.filterwarnings("ignore", category=DeprecationWarning)
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
job_name = '/0-1'
channels=['ele', 'mu', 'had']
sample = "EGamma"
year = '2018'

# Run the processor

In [23]:
# define the configuration and processor

fileset = {}
print(f"Processing: {sample}")
fileset[sample] = [f"../datafiles/{sample}/{sample}.root"]

outfiles = "./" + sample + "/"

# remove output directory to store fresh samples
if os.path.exists(outfiles):
    shutil.rmtree(outfiles)

from boostedhiggs.hwwprocessor import HwwProcessor
p = HwwProcessor(year=year, channels=channels, output_location=outfiles + job_name)

executor = processor.IterativeExecutor(status=True)
run = processor.Runner(
    executor=executor, savemetrics=True, schema=nanoevents.NanoAODSchema, chunksize=10000
)

out, metrics = run(
fileset, "Events", processor_instance=p
)

# dump to pickle
filehandler = open(outfiles + job_name + '.pkl', "wb")
pkl.dump(out, filehandler)
filehandler.close()

# merge parquet
for ch in channels:
    data = pd.read_parquet(outfiles + job_name + ch + '/parquet')
    data.to_parquet(outfiles + job_name + '_' + ch + '.parquet')

    # remove old parquet files
    os.system('rm -rf ' + outfiles + job_name + ch)

# remove old parquet files
os.system('rm -rf ' + outfiles + job_name)        

Processing: EGamma


Processing:   0%|          | 0/8 [00:00<?, ?chunk/s]

/opt/anaconda3/envs/coffea-env/lib/python3.7/site-packages/awkward/_connect/_numpy.py:210: RuntimeWarning: divide by zero encountered in log
  *[nplike.asarray(x) for x in inputs], **kwargs


# Load the output files

In [24]:
data = {}
metadata = {}

data[sample] = {}

outfiles = "./" + sample + "/"

for ch in channels:
    data[sample][ch] = pq.read_table(outfiles + job_name + '_' + ch + '.parquet').to_pandas()

with open(outfiles + job_name + '.pkl', 'rb') as f:
    metadata[sample] = pkl.load(f)

In [25]:
data['EGamma']['ele']

,lep_pt,lep_isolation,lep_misolation,lep_fj_m,lep_fj_bjets_ophem,lep_fj_dr,fj_msoftdrop,fj_pt,lep_met_mt,met,ht,trigger_iso,trigger_noiso,leptonInJet,anti_bjettag,anti_bjettag_ele,anti_bjettag_had
0,52.515980,0.004531,0.004531,-1.0,-1.000000,-1.000000,-1.00000,-1.000,19.889433,95.529259,253.218750,True,False,-1,-1,-1,-1
1,59.182159,0.081628,0.068446,-1.0,-1.000000,-1.000000,-1.00000,-1.000,95.808990,49.835381,220.375000,True,False,-1,-1,-1,-1
2,45.944492,0.038850,0.000000,-1.0,-1.000000,-1.000000,-1.00000,-1.000,32.184151,24.742369,227.000000,False,False,-1,-1,-1,-1
3,69.964195,0.000000,0.000000,-1.0,-1.000000,-1.000000,-1.00000,-1.000,7.056570,18.904827,230.687500,True,False,-1,-1,-1,-1
4,180.179260,0.007705,0.000000,NaN,0.031067,3.110147,15.59375,216.625,1.542515,61.541977,430.750000,True,True,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,103.342621,0.004677,0.000000,-1.0,-1.000000,-1.000000,-1.00000,-1.000,10.456084,115.595268,264.062500,True,False,-1,-1,-1,-1
530,49.254597,0.033731,0.026536,-1.0,-1.000000,-1.000000,-1.00000,-1.000,76.706108,61.162895,266.171875,True,False,-1,-1,-1,-1
531,102.173332,0.000000,0.000000,-1.0,-1.000000,-1.000000,-1.00000,-1.000,2.182878,17.412712,348.343750,True,False,-1,-1,-1,-1
532,42.951027,0.000000,0.024739,-1.0,-1.000000,-1.000000,-1.00000,-1.000,44.669979,12.251033,255.937500,True,False,-1,-1,-1,-1
